# Classifying game genres based on game descriptions
The goal is to use short (170 character) description of games to classify them as 1 og 31 game genres. The idea is that it should be possible to get a broad overview of what kind og game it is by reading the description. Let's have a few example:

>a game of exploration and survival in an epic fantasy setting

>A physics based game with 100 levels of challenging fun

>Welcome to the next-generation of open-world gaming

>The long awaited second expansion to Blizzards critically acclaimed RTS

>A Pay2Win business model simulator(adventure) that exposes the tricks online game businessmen use to maximize monetizatio

This illustrages some of the benifits and challanges of the dataset. Let's have a look at the 31 genres:

1. Action
2. Indie
3. Strategy
4. Early Access
5. Free to Play
6. Massively Multiplayer
7. RPG
8. Adventure
9. Casual
10. Simulation
11. Racing
12. Sports
13. **Audio Production**
14. **Utilities**
15. **Video Production**
16. **Education**
17. **Design & Illustration**
18. **Web Publishing**
19. **Photo Editing**
20. **Software Training**
21. **Animation & Modeling**
24. Puzzle
25. Platformer
26. Survival
27. Shooter
28. Horror
29. Sandbox
30. **Music**
31. Fighting
32. Hidden Objects
33. **Accounting**

Quite a lot of these (11 ~ 1/3) are not actually games. That is because Steam sell application that are not games.

To do this we will be using the sklearn python package. It is an extremely useful machine learning package that will allow us to build a powerfull classifier in just a couple of lines. The disadvantage of this package is that it makes it possible to build, very quickly, something that you don't understand. At best it is a problem solver, at worst it is a black box generator.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

## Getting and loading the data
The raw data are psql dumps from the homepage www.arcatetrail.com. The site is not longer active. 

The raw data is loaded using custom loading functions. 

In [3]:
import src.ReccomenderSystem as RC

titles = RC.reading_titles('data/game_titles.csv')
genres, num_games, num_attributes = RC.reading_votes('data/game_genres.csv')
id, taglines = RC.reading_taglines('data/game_tagline.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/game_titles.csv'

For example, one of the taglines is the following:

In [ ]:
print(taglines[1])

## Restructuring the data
Before we can use scikit-learn to build a classifier, we must make sure that the data is structured in a way scikit-learn will accept.

The classifier will take a string as imput and produce a numerical genre index. Below I make a list of all the game tags and a corresponding vector contining all the genre indices.

In [ ]:
y_mat = np.zeros((len(id),32))
X_mat = []
X_titles = []
kk = 0;
for ii in range(0,len(id)):
    tagline_index = np.where(genres[:,0]==int(id[ii]))
    if len(tagline_index[0])>0:
        X_mat.append(taglines[ii])
        for jj in range(0,len(tagline_index[0])):
            genre = int(genres[tagline_index[0][jj],1])
            y_mat[kk,genre-1] = 1
    else:
        X_mat.append(taglines[ii])
    kk = kk+1

## Exploring the data
Before I build the classifier, I it is usefull to explore the data to get a realistic expection about how successful the classifier can be. Below I look at how the games are distributed across the different genres.

In [ ]:
aggregate = sum(y_mat)
plt.bar(range(len(aggregate)),aggregate)
plt.show()

Many of the genres are empty. That is, there are no games with that genre. There are also quite a few of the genres that have less than 50 games in them. It will be difficult to build a good classifier for there.

## Building my own word tokenizer to take care of stemming
I am not sure this is actually used.

In [ ]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

## Building the classifier using Multinomial Naive Bayes

In [ ]:
genre_to_train = 0;
train_size = int(round(len(y_mat)*0.8))

In [ ]:
y_train = y_mat[0:train_size,genre_to_train]
X_train = X_mat[0:train_size]

In [ ]:
clf = Pipeline([
('vect', TfidfVectorizer(stop_words='english',smooth_idf=1,use_idf=1,analyzer='word')),
('tfidf', TfidfTransformer(norm='l2',use_idf=True,smooth_idf=True)),
('clf', MultinomialNB())
])
clf = clf.fit(X_train, y_train)

In [ ]:
X_test = X_mat[train_size:-1];
y_test = y_mat[train_size:-1,genre_to_train];
MultinomialNB_predicted = clf.predict(X_test)

print(metrics.classification_report(y_test, MultinomialNB_predicted))

So answer this question I will loop over all the genres and train and evaluate the classifier. I will save the average precision, recall and f1 scores.

In [ ]:
recall = np.zeros(32)
for genre in range(0,32):
    y_train = y_mat[0:train_size,genre]
    X_train = X_mat[0:train_size]
    X_test = X_mat[train_size:-1];
    y_test = y_mat[train_size:-1,genre];
    if sum(y_train) == 0:
        print('No training cases in this class')
    elif sum(y_test) == 0:
        print('No true samples in the test set')
    else:
        MultinomialNB_predicted = clf.predict(X_test)
        recall[genre] = metrics.recall_score(y_test, MultinomialNB_predicted)

In [ ]:
plt.bar(range(0,32),recall)
plt.show()

So quite a few of the genres have 0 recall. This is because there are in fact no gemres in the test set with those genres. If we exclude those we end up with an average recall of;

In [ ]:
print(sum(recall)/sum(recall>0))

Which is not great, but its not so bad etiher. 

From here several things could be done:
- improve the quality and quantity of the data
- Use different and more specific genres
- Build better classifier, perhaps tailoring the classifier for each genre
- Use more features. 

## Trying out Support Vector as classifier

In [ ]:
clf_SV = Pipeline([
('vect', CountVectorizer(stop_words='english')),
('tfidf', TfidfTransformer()),
('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=10, random_state=42))
])

recall_SV = np.zeros(32)
for genre in range(0,32):
    y_train = y_mat[0:train_size,genre]
    X_train = X_mat[0:train_size]
    X_test = X_mat[train_size:-1];
    y_test = y_mat[train_size:-1,genre];
    if sum(y_train) == 0:
        print('No training cases in this class')
    else:
        clf_SV = clf_SV.fit(X_train, y_train)
        SV_predicted = clf_SV.predict(X_test)
        recall_SV[genre] = metrics.recall_score(y_test, SV_predicted)

In [ ]:
print(sum(recall_SV)/sum(recall_SV>0))

In [ ]:
plt.bar(range(0,32),recall_SV)
plt.show()

## Cool applications of the classifier described above
- classify.me: A website that classifies a game given a tagline.
- tagline generator: One could run the algorithm in reverse. Given a genre, generate a tagline. I'm not entirely sure how one would do this. The idea is to get some of the top keywords from a genre and then use those to generate a short description. I'm sure other people have worked with generating sentences string from keywords. See eg. [Narrative Science](#https://www.narrativescience.com/) 